In [1]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [2]:
import os
import shutil

In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key="yccesU5wFOObLtdMbTzW")
project = rf.workspace("lscm").project("gld-pallet-detection-for-yolov5-1.0")
version = project.version(7)
dataset = version.download("yolov5")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to GLD-Pallet-detection-For-YOLOv5-1.0-7 in yolov5pytorch:: 100%|██████████| 3724/3724 [00:02<00:00, 1856.54it/s]


In [4]:
if not os.path.exists("cube_detector"):
    os.system("git clone https://github.com/thohemp/cube_detector.git")

In [ ]:
!cat cube_detector/yolov5/data/cube.yaml

path: /content/GLD-Pallet-detection-For-YOLOv5-1.0-7
train: train/images
val: valid/images
test: test/images

# Classes
nc: 3
names: ['feet', 'pallet', 'pallet_s']

In [5]:
source_path = "/content/GLD-Pallet-detection-For-YOLOv5-1.0-7"
!cat /content/GLD-Pallet-detection-For-YOLOv5-1.0-7/data.yaml

!cp /content/GLD-Pallet-detection-For-YOLOv5-1.0-7/data.yaml cube_detector/yolov5/data/cube.yaml

with open("cube_detector/yolov5/data/cube.yaml", "w") as f:
    f.write("""path: /content/GLD-Pallet-detection-For-YOLOv5-1.0-7
train: train/images
val: valid/images
test: test/images

# Classes
nc: 3
names: ['feet', 'pallet', 'pallet_s']""")

names:
- feet
- pallet
- pallet_s
nc: 3
roboflow:
  license: CC BY 4.0
  project: gld-pallet-detection-for-yolov5-1.0
  url: https://universe.roboflow.com/lscm/gld-pallet-detection-for-yolov5-1.0/dataset/7
  version: 7
  workspace: lscm
test: ../test/images
train: GLD-Pallet-detection-For-YOLOv5-1.0-7/train/images
val: GLD-Pallet-detection-For-YOLOv5-1.0-7/valid/images


In [6]:
!cat cube_detector/yolov5/data/cube.yaml

path: /content/GLD-Pallet-detection-For-YOLOv5-1.0-7
train: train/images
val: valid/images
test: test/images

# Classes
nc: 3
names: ['feet', 'pallet', 'pallet_s']

In [7]:
!wget https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5s.pt -O yolov5/yolov5s.pt

yolov5/yolov5s.pt: No such file or directory


In [8]:
with open("/content/cube_detector/yolov5/models/common.py", "a") as f:
    f.write("""
class SPPF(nn.Module):
    # Spatial Pyramid Pooling - Fast (SPPF) layer for YOLOv5
    def __init__(self, c1, c2, k=5):  # equivalent to SPP(k=(5, 9, 13))
        super().__init__()
        c_ = c1 // 2  # hidden channels
        self.cv1 = Conv(c1, c_, 1, 1)
        self.cv2 = Conv(c_ * 4, c2, 1, 1)
        self.m = nn.MaxPool2d(kernel_size=k, stride=1, padding=k // 2)

    def forward(self, x):
        x = self.cv1(x)
        return self.cv2(torch.cat((x, self.m(x), self.m(self.m(x)), self.m(self.m(self.m(x)))), 1))
    """)

In [9]:
!cat cube_detector/yolov5/models/yolov5s.yaml

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, Fa

In [10]:
with open("/content/cube_detector/yolov5/models/yolov5s.yaml", "w") as f:
    f.write("""
# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPPF, [1024, 1024, 5]],  # Updated channels
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]
""")


In [11]:
!rm -rf /content/cube_detector/yolov5
!git clone https://github.com/ultralytics/yolov5.git /content/cube_detector/yolov5

Cloning into '/content/cube_detector/yolov5'...
remote: Enumerating objects: 17067, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 17067 (delta 24), reused 28 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17067/17067), 15.68 MiB | 16.81 MiB/s, done.
Resolving deltas: 100% (11714/11714), done.


In [ ]:
import os
import shutil
import glob

# Define paths
original_train_path = "/content/GLD-Pallet-detection-For-YOLOv5-1.0-7/train"
original_valid_path = "/content/GLD-Pallet-detection-For-YOLOv5-1.0-7/valid"
original_test_path = "/content/GLD-Pallet-detection-For-YOLOv5-1.0-7/test"

limited_train_path = "/content/limited_dataset/train"
limited_valid_path = "/content/limited_dataset/valid"
limited_test_path = "/content/limited_dataset/test"

# Create directories for train, valid, and test
for path in [limited_train_path, limited_valid_path, limited_test_path]:
    os.makedirs(os.path.join(path, "images"), exist_ok=True)
    os.makedirs(os.path.join(path, "labels"), exist_ok=True)

# Helper function to copy files
def copy_files(src_path, dest_path, max_boxes=3):
    labels_src = os.path.join(src_path, "labels")
    images_src = os.path.join(src_path, "images")
    labels_dest = os.path.join(dest_path, "labels")
    images_dest = os.path.join(dest_path, "images")

    for label_file in glob.glob(os.path.join(labels_src, "*.txt")):
        with open(label_file, "r") as f:
            lines = f.readlines()
        if len(lines) <= max_boxes:  # Only include images with <= max_boxes
            shutil.copy(label_file, labels_dest)
            image_file = label_file.replace("/labels/", "/images/").replace(".txt", ".jpg")
            shutil.copy(image_file, images_dest)

# Copy data for train, valid, and test datasets
copy_files(original_train_path, limited_train_path, max_boxes=3)
copy_files(original_valid_path, limited_valid_path, max_boxes=3)
copy_files(original_test_path, limited_test_path, max_boxes=3)

print("Train, validation, and test datasets have been moved to the limited dataset.")


Train, validation, and test datasets have been moved to the limited dataset.


In [16]:
# Define the new configuration
config_content = """
path: /content/GLD-Pallet-detection-For-YOLOv5-1.0-7/
train: train/images
val: valid/images
test: test/images
nc: 3
names: ['feet', 'pallet', 'pallet_s']
"""

# Write the new configuration to the cube.yaml file
with open("/content/cube_detector/yolov5/data/cube.yaml", "w") as f:
    f.write(config_content)

print("Dataset configuration updated successfully.")

Dataset configuration updated successfully.


In [17]:
!python cube_detector/yolov5/train.py --img 640 --batch 16 --epochs 25 --data cube_detector/yolov5/data/cube.yaml --weights cube_detector/yolov5/yolov5s.pt

Streaming output truncated to the last 5000 lines.
       1/24       4.4G    0.08248    0.02903    0.01521         56        640:  34% 35/102 [00:15<00:29,  2.27it/s]/content/cube_detector/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
       1/24       4.4G    0.08233    0.02893    0.01513         79        640:  35% 36/102 [00:15<00:25,  2.55it/s]/content/cube_detector/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
       1/24       4.4G    0.08223    0.02882    0.01504         86        640:  36% 37/102 [00:15<00:26,  2.47it/s]/content/cube_detector/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocas

In [24]:
import torch
import os
os.chdir('/content/cube_detector/yolov5')

weights_path = "/content/cube_detector/yolov5/runs/train/exp2/weights/best.pt"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_data = torch.load(weights_path, map_location=device)

# Inspect the keys in the loaded weights file
print(model_data.keys())

# Optional: Inspect specific keys, like 'model'
if 'model' in model_data:
    print(model_data['model'])

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


dict_keys(['epoch', 'best_fitness', 'model', 'ema', 'updates', 'optimizer', 'opt', 'git', 'date'])
DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, mo

In [25]:
!pip install -r /content/cube_detector/yolov5/requirements.txt

In [30]:
import time
import torch
from cube_detector.yolov5.models.common import DetectMultiBackend
from cube_detector.yolov5.utils.dataloaders import LoadImages
from cube_detector.yolov5.utils.general import non_max_suppression
from cube_detector.yolov5.utils.torch_utils import select_device, time_sync

# Paths
weights_path = "/content/cube_detector/yolov5/runs/train/exp2/weights/best.pt"  # Trained model
test_images_path = "/content/GLD-Pallet-detection-For-YOLOv5-1.0-7/test/images"  # Test images directory

# Initialize model
device = select_device('')  # Use GPU if available, else CPU
model = DetectMultiBackend(weights_path, device=device)
stride, names = model.stride, model.names
img_size = 640  # Image size used during training

# Load test images
dataset = LoadImages(test_images_path, img_size=img_size, stride=stride, auto=True)

# Inference on a single image
print("Testing single image inference speed...")
for path, img, im0s, vid_cap, s in dataset:
    img = torch.from_numpy(img).to(device).float() / 255.0  # Normalize
    img = img.unsqueeze(0)  # Add batch dimension

    # Warm up model
    model.warmup(imgsz=(1, 3, img_size, img_size))  # Correct warmup method

    # Measure inference time
    t1 = time_sync()
    pred = model(img)
    t2 = time_sync()

    # Apply NMS (optional)
    pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, max_det=5)

    print(f"Inference time for single image: {t2 - t1:.4f} seconds")
    break  # Test only the first image

# Inference on a batch of images
print("\nTesting batch inference speed...")
batch_images = []
for path, img, im0s, vid_cap, s in dataset:
    img = torch.from_numpy(img).to(device).float() / 255.0  # Normalize
    batch_images.append(img)
    if len(batch_images) == 16:  # Use a batch of 16 images
        break

batch_images = torch.stack(batch_images, dim=0)  # Create batch tensor
t1 = time_sync()
pred = model(batch_images)
t2 = time_sync()

print(f"Inference time for a batch of 16 images: {t2 - t1:.4f} seconds")
print(f"Average inference time per image in batch: {(t2 - t1) / 16:.4f} seconds")

# Calculate FPS (Frames Per Second)
fps = 1 / ((t2 - t1) / 16)
print(f"Frames Per Second (FPS): {fps:.2f}")


YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs


Testing single image inference speed...
Inference time for single image: 0.0114 seconds

Testing batch inference speed...
Inference time for a batch of 16 images: 0.1328 seconds
Average inference time per image in batch: 0.0083 seconds
Frames Per Second (FPS): 120.53


In [34]:
# Ensure img is a torch.Tensor and normalized
img = img.to(device).float() / 255.0  # Normalize to [0, 1]
img = img.unsqueeze(0)  # Add batch dimension: (C, H, W) -> (1, C, H, W)

# Model inference
pred = model(img)  # Forward pass

# Apply Non-Max Suppression (NMS) to filter predictions
pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, max_det=5)

# Print results
print(f"Prediction: {pred}")


Prediction: [tensor([], device='cuda:0', size=(0, 6))]


In [35]:
for det in pred:  # Iterate through detections for each image
    if det is not None and len(det):  # If there are detections
        for *xyxy, conf, cls in det:
            x1, y1, x2, y2 = map(int, xyxy)  # Convert to integers
            confidence = float(conf)  # Object confidence
            class_id = int(cls)  # Class ID
            print(f"Class {class_id}, Confidence: {confidence}, Box: ({x1}, {y1}), ({x2}, {y2})")